https://github.com/projectashik/nepali-number-validator

In [1]:
import re

def validate_nepali_phone(phone_number):
    """
    नेपाली फोन नम्बर (मोबाइल र ल्यान्डलाइन) प्रमाणीकरण गर्दछ।
    - NTC, Ncell, र Smart Cell मोबाइल नम्बरहरूलाई समर्थन गर्दछ।
    - +977 देश कोड, स्पेस, वा ड्यास सहितको नम्बरहरू स्वीकार गर्दछ।
    
    Args:
        phone_number (str): प्रमाणीकरण गर्नुपर्ने फोन नम्बर।
        
    Returns:
        dict: नम्बरको वैधता र विवरण सहितको जानकारी।
    """
    # १. नम्बरलाई सफा गर्ने (अनावश्यक चिन्ह हटाउने)
    # स्पेस, ड्यास, कोष्ठक जस्ता चिन्हहरू हटाउने
    sanitized_number = re.sub(r'[\s\-()]', '', str(phone_number))

    # २. देशको कोड (+977 वा 977) हटाउने
    if sanitized_number.startswith('+977'):
        local_number = sanitized_number[4:]
    elif sanitized_number.startswith('977'):
        local_number = sanitized_number[3:]
    else:
        local_number = sanitized_number

    # ३. अपरेटरको प्रिफिक्सहरू परिभाषित गर्ने
    operators = {
        "नेपाल टेलिकम (NTC)": ("984", "985", "986", "974", "975", "976"),
        "एनसेल (Ncell)": ("980", "981", "982", "970", "971", "972"),
        "स्मार्ट सेल (Smart Cell)": ("961", "962", "963", "988"),
    }

    # ४. प्रमाणीकरण तर्क
    # मोबाइल नम्बर जाँच (१० अंकको हुनुपर्छ)
    if local_number.isdigit() and len(local_number) == 10:
        for operator, prefixes in operators.items():
            if local_number.startswith(prefixes):
                return {
                    "is_valid": True,
                    "number": local_number,
                    "type": "मोबाइल",
                    "operator": operator,
                    "message": f"यो एक मान्य '{operator}' मोबाइल नम्बर हो।",
                }

    # ल्यान्डलाइन नम्बर जाँच (० बाट सुरु भई ८ देखि १० अंकको)
    # उदाहरण: 014412345 (९ अंक), 023580123 (९ अंक)
    landline_pattern = r'^0\d{7,9}$'
    if re.match(landline_pattern, local_number):
         return {
            "is_valid": True,
            "number": local_number,
            "type": "ल्यान्डलाइन",
            "operator": "नेपाल टेलिकम (ल्यान्डलाइन)",
            "message": "यो एक मान्य ल्यान्डलाइन नम्बर हो।",
        }

    # ५. यदि कुनै ढाँचामा मिलेन भने, नम्बर अमान्य हुन्छ
    return {
        "is_valid": False,
        "number": phone_number,
        "type": "अज्ञात",
        "operator": "N/A",
        "message": "यो एक अमान्य नेपाली फोन नम्बर हो।",
    }


# --- उदाहरणहरू ---

# परीक्षणका लागि फोन नम्बरहरूको सूची
test_numbers = [
    "9841234567",         # मान्य NTC
    "+977 980-8123456",   # मान्य Ncell (देश कोड, स्पेस र ड्यास सहित)
    "9610012345",         # मान्य Smart Cell
    "014412345",          # मान्य काठमाडौं ल्यान्डलाइन
    "9876543210",         # अमान्य प्रिफिक्स
    "984112233",          # अमान्य (अंक नपुग)
    "यो नम्बर होइन",       # अमान्य (अक्षरहरू)
    "+9779741234567",     # मान्य NTC (देश कोड सहित)
    "056-520123"          # मान्य चितवन ल्यान्डलाइन
]

print("======= नेपाली फोन नम्बर प्रमाणीकरणकर्ता =======")
print("-" * 50)

# सूचीमा भएका सबै नम्बरहरूलाई एक-एक गरी प्रमाणीकरण गर्ने
for number in test_numbers:
    result = validate_nepali_phone(number)
    print(f"दिइएको नम्बर: '{number}'")
    print(f"परिणाम: {result['message']}")
    if result['is_valid']:
        print(f"  -> प्रकार: {result['type']}")
        print(f"  -> अपरेटर: {result['operator']}")
        print(f"  -> सफा नम्बर: {result['number']}")
    print("-" * 50)

======= नेपाली फोन नम्बर प्रमाणीकरणकर्ता =======
--------------------------------------------------
दिइएको नम्बर: '9841234567'
परिणाम: यो एक मान्य 'नेपाल टेलिकम (NTC)' मोबाइल नम्बर हो।
  -> प्रकार: मोबाइल
  -> अपरेटर: नेपाल टेलिकम (NTC)
  -> सफा नम्बर: 9841234567
--------------------------------------------------
दिइएको नम्बर: '+977 980-8123456'
परिणाम: यो एक मान्य 'एनसेल (Ncell)' मोबाइल नम्बर हो।
  -> प्रकार: मोबाइल
  -> अपरेटर: एनसेल (Ncell)
  -> सफा नम्बर: 9808123456
--------------------------------------------------
दिइएको नम्बर: '9610012345'
परिणाम: यो एक मान्य 'स्मार्ट सेल (Smart Cell)' मोबाइल नम्बर हो।
  -> प्रकार: मोबाइल
  -> अपरेटर: स्मार्ट सेल (Smart Cell)
  -> सफा नम्बर: 9610012345
--------------------------------------------------
दिइएको नम्बर: '014412345'
परिणाम: यो एक मान्य ल्यान्डलाइन नम्बर हो।
  -> प्रकार: ल्यान्डलाइन
  -> अपरेटर: नेपाल टेलिकम (ल्यान्डलाइन)
  -> सफा नम्बर: 014412345
--------------------------------------------------
दिइएको नम्बर: '9876543210'
परिणाम: यो